# AutoQUBO Portfolio Optimisation

In [10]:
import csv
import numpy as np
import dynex
from autoqubo import Binarization, SamplingCompiler, SearchSpace, Utils

## Import Portfolio data

In [4]:
with open('data/portfolio.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    n, budget = map(int, next(csv_reader))
    cov_matrix = np.zeros((n, n))
    mean_vector = np.zeros(n)
    for row in csv_reader:
        if len(row) == 2:
            i, v = map(int, row)
            mean_vector[i] = v
        elif len(row) == 3:
            i, j, v = map(int, row)
            cov_matrix[i, j] = v
        else:
            raise ValueError

In [5]:
print("means:", mean_vector)
print("cov:")
print(cov_matrix)

means: [1. 2. 3.]
cov:
[[3. 1. 3.]
 [0. 4. 4.]
 [0. 0. 2.]]


## Define support functions:

In [6]:
def variance(x):
    """
    Variance
    """
    return x@cov_matrix@x


def mean(x):
    """
    Mean return
    """
    return x@mean_vector


def constraint(x):
    """
    Budget constraint
    """
    return (x.sum() - budget)**2

## Optimisation function:

In [7]:
A, B, C = 1, -1, 100

In [29]:
# Mean-variance portfolio optimization model
def f(x):
    return A*variance(x) + B*mean(x) + C*constraint(x)


## Autogenerate Qubo model:

In [31]:
s = SearchSpace()
weights_vector = Binarization.get_uint_vector_type(3, 3)
s.add('x', weights_vector, 3 * 3)

qubo, offset = SamplingCompiler.generate_qubo_matrix(fitness_function=f, input_size=s.size, searchspace=s, use_multiprocessing=False)

## Is generated Qubo model valid?

In [32]:
if SamplingCompiler.test_qubo_matrix(f, qubo, offset, search_space=s):
    print("QUBO generation successful")
else:
    print("QUBO generation failed - the objective function is not quadratic")

QUBO generation successful


In [33]:
print("QUBO matrix:")
print(qubo)
print("QUBO offset")
print(f"x[] = {offset}")

QUBO matrix:
[[ -698.   412.   824.   201.   402.   804.   203.   406.   812.]
 [    0. -1190.  1648.   402.   804.  1608.   406.   812.  1624.]
 [    0.     0. -1556.   804.  1608.  3216.   812.  1624.  3248.]
 [    0.     0.     0.  -698.   416.   832.   204.   408.   816.]
 [    0.     0.     0.     0. -1188.  1664.   408.   816.  1632.]
 [    0.     0.     0.     0.     0. -1544.   816.  1632.  3264.]
 [    0.     0.     0.     0.     0.     0.  -701.   408.   816.]
 [    0.     0.     0.     0.     0.     0.     0. -1198.  1632.]
 [    0.     0.     0.     0.     0.     0.     0.     0. -1580.]]
QUBO offset
x[] = 1600.0


## Sample on Dynex:

In [35]:
print("Best solutions (minimize):")
sampleset = dynex.sample_qubo(qubo, offset, mainnet=False, num_reads=1024, annealing_time=200)
print(sampleset)

Best solutions (minimize):
[DYNEX] PRECISION SET TO 0.1
[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬─────────────┬───────────┬───────────────────────────┬─────────┬─────────┬────────────────╮
│   DYNEXJOB │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CHIPS   │ STEPS   │ GROUND STATE   │
├────────────┼─────────────┼───────────┼───────────────────────────┼─────────┼─────────┼────────────────┤
│         -1 │           0 │           │ *** WAITING FOR READS *** │         │         │                │
╰────────────┴─────────────┴───────────┴───────────────────────────┴─────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.00 SECONDS
[DYNEX] SAMPLESET READY
   0  1  2  3  4  5  6  7  8 energy num_oc.
0  1  0  0  0  0  0  1  1  0   20.0       1
['BINARY', 1 rows, 1 samples, 9 variables]


## Output optimal solution:

In [36]:
sol = sampleset.record[0][0]
x = s.decode_dict(sol)['x']
e = sampleset.first.energy
print(
    f"x={x}, "
    f"energy={e}, "
    f"obj={variance(x)-mean(x)}, "
    f"constraint={constraint(x)}"
)

x=[1 0 3], energy=20.0, obj=20.0, constraint=0
